In [ ]:
import datetime

print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

In [ ]:
import life
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from pathlib import Path

reports = life.get_reports_cached()

In [ ]:
def report_to_dict(x: life.GetPychiatristReport):
    ret = {}
    ret["Date"] = x.Date
    ret["Anxiety"] = x.Anxiety.scale_1_to_10
    ret["Happiness"] = x.Happiness.scale_1_to_10
    # ret["Depression"] = x.Depression.scale_1_to_10
    # ret["Satisfication"] = x.Satisfication.scale_1_to_10
    # ret["Mania"] =  x.Mania.scale_1_to_10
    return ret


# sample.Anxiety.scale_1_to_10

# set default figure size for matplotlib
plt.rcParams["figure.figsize"] = (20, 5)
df_original = pd.DataFrame.from_records([report_to_dict(r) for r in reports])

In [ ]:
# df = df_original[df_original ["Date"] < "2015-01-01"]
df = df_original
df = df.set_index("Date")
# smooth out the data
# df = df.rolling(7).mean()
# resample the data with a mean

time_unit = "M"
df_unit_time = (
    df.resample(time_unit)
    .count()
    .rename(columns={"Anxiety": "Count"})
    .drop(columns=["Happiness"])
)
# unit time is upto 30, so divide by 30 and convert to an int percentage
df_unit_time["Percent"] = (100 * df_unit_time["Count"] / 30).astype(int)
df_unit_time.plot(title=f"Compliance by {time_unit}", y="Percent")

# add veritcal lines for major for every year
for year in range(2013, 2024):
    plt.axvline(x=f"{year}-01-01", color="red", linestyle="--")

# Add horizontal lines for 2,4,6,8
for i in [25, 50, 75, 100]:
    plt.axhline(y=i, color="green", linestyle="--")


df = df.resample(time_unit).mean()
df.rolling(3).mean().plot(title=f"Mental Health Over Time by {time_unit}")

for year in range(2013, 2024):
    plt.axvline(x=f"{year}-01-01", color="red", linestyle="--")

for i in range(3, 8, 1):
    plt.axhline(y=i, color="green", linestyle="--")


df = df.resample(time_unit).mean()
df.rolling(2).mean().plot(title=f"Happiness Over Time by {time_unit}", y="Happiness")

for year in range(2013, 2024):
    plt.axvline(x=f"{year}-01-01", color="red", linestyle="--")

for i in range(4, 8, 1):
    plt.axhline(y=i, color="green", linestyle="--")

df = df.resample(time_unit).mean()
df.rolling(2).mean().plot(title=f"Anxiety Over Time by {time_unit}", y="Anxiety")


# add veritcal lines for major for every year
for year in range(2013, 2024):
    plt.axvline(x=f"{year}-01-01", color="red", linestyle="--")

for i in range(3, 7, 1):
    plt.axhline(y=i, color="green", linestyle="--")

In [ ]:
px.line(df, y="Happiness")
df = px.data.gapminder().query("country=='Canada'")
fig = px.line(df, x="year", y="lifeExp", title="Life expectancy in Canada")
fig.show()

In [ ]:
# Lets repivot just happiness
df = df_original.set_index("Date")
df = df.resample("W").mean()
df = df.rolling(3, center=True).mean()  # TODO TWEAK HOW W EDO THE Smoothing
df["Year"] = df.index.year
df["Week"] = df.index.isocalendar().week

# Drop any metric below 1
df = df[df.Happiness > 1]  # Drop outliers
df = df[df.Anxiety > 1]  # Drop outliers

# df["Day"] = df.index.dayofyear
# df[df.Year == 2022].sort_values
# Last years at Meta
df = df[df.Year > 2012]
df = df[df.Year < 2017]


metric = "Anxiety"
df2 = df.pivot_table(index="Week", columns="Year", values=metric)
df2.plot(title=metric)

# add veritcal lines for major for every year
for week in [13, 26, 39]:
    plt.axvline(x=week, color="red", linestyle="--")

for i in range(3, 7, 1):
    plt.axhline(y=i, color="green", linestyle="--")

# df2

In [ ]:
def report_to_dict(x: life.GetPychiatristReport):
    ret = {}
    ret["Date"] = x.Date
    ret["Anxiety"] = x.Anxiety.scale_1_to_10
    ret["Happiness"] = x.Happiness.scale_1_to_10
    ret["Depression"] = x.Depression.scale_1_to_10
    ret["Satisfication"] = x.Satisfication.scale_1_to_10
    # ret["Mania"] =  x.Mania.scale_1_to_10
    return ret


# sample.Anxiety.scale_1_to_10

# set default figure size for matplotlib
# df_original =  pd.DataFrame.from_records([report_to_dict(r) for r in reports])
r = reports[380]
[p.Name for p in r.PeopleInEntry]

In [ ]:
rs = [{"date": r.Date, "People": [p.Name for p in r.PeopleInEntry]} for r in reports]
rs[20]


def to_person_sentiment(r):
    row = {"date": r.Date}
    for p in r.PeopleInEntry:
        row[p.Name] = p.Sentiment.lower()
    return row


rs = [to_person_sentiment(r) for r in reports]

In [ ]:
# Create a dataframe from the rs list
df = pd.DataFrame(rs)

# fitler to post Meta
df = df.loc[(df.date > "2011") & (df.date < "2013")]
# df = df.loc[df.date == "2013"]
display(df)

# Get columns iwht the most data.
na_count = df.isna().sum()
# Sort the columns by number of missing values and select the top 30
top_30_columns = na_count.nsmallest(30).index

# Select only the top 30 columns in the DataFrame
df_filtered = df[top_30_columns]

print(f"From: {df.date.min()} to {df.date.max()}")
for column in df_filtered.columns[1:]:
    print(f"For {column}")
    print(df_filtered[column].value_counts(normalize=True)[:5])

In [ ]:
df = pd.DataFrame([life.to_people_sentiment_dict(r) for r in reports]).set_index("date")

In [ ]:
df

In [ ]:
df.tori.value_counts()

In [ ]:
def g1(r):
    nPositive = len(r[r == "positive"])
    # nNegative = len(r[r == "negative"])
    nNeutral = len(r[r == "neutral"]) + len(r[r == "mixed"])
    nTotal = len(r)
    return nPositive / (nTotal - nNeutral)


df.groupby(pd.Grouper(freq="Y"))["pablo"].apply(g1).plot()
# df.groupby(pd.Grouper(freq="Y"))["tori"].value_counts()

# for (key,group) in df.groupby(pd.Grouper(freq="Y"))["tori"]:
#   ic(group.value_counts())

In [ ]:
df_messages = pd.read_pickle(Path.home() / "gits/nlp/df_messages.pickle.zip")

In [ ]:
df_messages.columns

In [ ]:
df_messages.to_phone.value_counts()

In [ ]:
df_messages[df_messages.to_phone == "+12065383855"]

In [ ]:
# Messsages I sent, but uniquify group messages to avoid double counts
# df_messages.set_index('date',inplace=True)
df_sent_messages = df_messages[df_messages["is_from_me"] == 1].drop_duplicates(
    subset="text"
)

# Only Keep messages from senders I send to post
top_receivers = df_sent_messages.to_phone.value_counts().head(10).index[2:]
df_sent_messages = df_sent_messages[df_sent_messages.to_phone.isin(top_receivers)]

monthly_counts = (
    df_sent_messages.groupby([pd.Grouper(freq="M"), "to_phone"])
    .size()
    .unstack(fill_value=0)
)
monthly_counts.plot()

In [ ]:
df_sent_messages.set_index("date").resample("M").size().plot()

In [ ]:
monthly_counts = (
    df_sent_messages.set_index("date")
    .groupby([pd.Grouper(freq="M"), "to_phone"])
    .size()
)  # .unstack(level=[1,2], fill_value=0)

In [ ]:
monthly_counts

In [ ]:
monthly_counts.unstack()

In [ ]:
df_sent_messages = df_messages[df_messages["is_from_me"] == 1].drop_duplicates(
    subset="text"
)
# df_sent_messages = df_sent_messages.set_index("date")
df_sent_messages.resample("M").size().loc["2020"].plot()